<a href="https://colab.research.google.com/github/joannafernandez/cnt_MSci/blob/main/L5%266_MAPQ_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mapping Quality (MAPQ) and Filtering Alignments

In this worksheet you will learn:
- What mapping quality (MAPQ) is
- How Bowtie2 assigns MAPQ scores
- How to inspect MAPQ in BAM files
- How filtering affects downstream analysis
- Why filtering is essential in genomics workflows

This notebook uses a **toy genome and small BAM file** so results are easy to interpret.


## Prerequisites

This notebook assumes you have:
- A BAM file produced by Bowtie2
- `samtools` installed

If not, we will generate a small example BAM below.


Install tools (safe to rerun)

In [ ]:
!apt-get update -qq
!apt-get install -y samtools bowtie2


## Set up working directories

In [ ]:
%%bash
set -e
cd ~
mkdir -p mapq_demo/{genome,reads,alignments}
cd mapq_demo
pwd

## What is Mapping Quality (MAPQ)?

**Mapping Quality (MAPQ)** is a score that estimates how confident the aligner is that
a read has been placed in the correct genomic location.

Important points:
- MAPQ is **not** base quality
- MAPQ is assigned **per alignment**
- Higher MAPQ = higher confidence
- MAPQ = 0 often means:
  - The read maps equally well to multiple locations

Each aligner calculates MAPQ differently.

## Why repeats matter

Repeats cause **ambiguous alignments**.
We will create a genome with repeated sequences to demonstrate this.

In [ ]:
%%bash
set -e
cd ~/mapq_demo/genome

cat > repeat_genome.fa << 'EOF'
>chrToy
ATGCGTACGTAGCTAGCTAGCTAGCTAGCTGACTGACTGACTGACTGAC
ATGCGTACGTAGCTAGCTAGCTAGCTAGCTGACTGACTGACTGACTGAC
ATGCGTACGTAGCTAGCTAGCTAGCTAGCTGACTGACTGACTGACTGAC
EOF

bowtie2-build repeat_genome.fa repeat_index

## Create reads that map ambiguously

Some reads will match **multiple locations** equally well.

In [ ]:
%%bash
set -e
cd ~/mapq_demo/reads

cat > reads_R1.fastq << 'EOF'
@read1/1
ATGCGTACGTAGCTAGCTA
+
IIIIIIIIIIIIIIIIIII
@read2/1
GCTAGCTAGCTAGCTAGCT
+
IIIIIIIIIIIIIIIIIII
EOF

cat > reads_R2.fastq << 'EOF'
@read1/2
TAGCTAGCTAGCTGACTGA
+
IIIIIIIIIIIIIIIIIII
@read2/2
CTAGCTAGCTAGCTGACTG
+
IIIIIIIIIIIIIIIIIII
EOF

## Align reads to the repetitive genome

In [ ]:
%%bash
set -e
cd ~/mapq_demo

bowtie2 \
  -x genome/repeat_index \
  -1 reads/reads_R1.fastq \
  -2 reads/reads_R2.fastq \
  -S alignments/aligned.sam

## Convert SAM to sorted BAM


In [ ]:
%%bash
set -e
cd ~/mapq_demo/alignments

samtools view -bS aligned.sam > aligned.bam
samtools sort aligned.bam -o aligned.sorted.bam
samtools index aligned.sorted.bam


## Inspect MAPQ values

MAPQ is stored in **column 5** of a SAM file.

In [ ]:
!samtools view ~/mapq_demo/alignments/aligned.sorted.bam

## Examine MAPQ distribution

We will count how many reads have each MAPQ score.

In [ ]:
%%bash
set -e
cd ~/mapq_demo/alignments

samtools view aligned.sorted.bam | awk '{print $5}' | sort | uniq -c


## Filtering by MAPQ

Common thresholds:
- MAPQ ≥ 20 : reasonably confident
- MAPQ ≥ 30 : high confidence
- MAPQ = 0  : ambiguous (often discarded)

Filtering removes unreliable alignments **before** downstream analysis.

In [ ]:
%%bash
set -e
cd ~/mapq_demo/alignments

samtools view -b -q 20 aligned.sorted.bam > aligned.mapq20.bam
samtools index aligned.mapq20.bam

echo "Before filtering:"
samtools flagstat aligned.sorted.bam

echo ""
echo "After MAPQ ≥ 20 filtering:"
samtools flagstat aligned.mapq20.bam


## Why MAPQ filtering matters

Low-MAPQ reads can:
- Inflate signal in repetitive regions
- Create false peaks
- Distort coverage tracks
- Mislead biological interpretation

Most pipelines apply MAPQ filtering early.


## Common real-world BAM filters

Typical filters combined together:

- Remove unmapped reads
- Remove secondary alignments
- Remove supplementary alignments
- Filter by MAPQ

Example:


In [ ]:
%%bash
set -e
cd ~/mapq_demo/alignments

samtools view -b \
  -q 30 \
  -F 4 \
  -F 256 \
  -F 2048 \
  aligned.sorted.bam > aligned.clean.bam

samtools index aligned.clean.bam
samtools flagstat aligned.clean.bam


## Important SAM flags

| Flag | Meaning |
|-----:|--------|
| 4 | Unmapped |
| 256 | Secondary alignment |
| 2048 | Supplementary alignment |
| 1024 | Duplicate |

Flags are combined using `-F` to exclude them.


## Exercises

1. Filter reads with MAPQ ≥ 10 and ≥ 30 — compare counts
2. Remove only MAPQ 0 reads — what remains?
3. Inspect SAM lines before and after filtering
4. Why might MAPQ filtering differ for:
   - Histone marks
   - Transcription factors


## Key takeaways

- MAPQ measures alignment confidence, not base quality
- Repeats lower MAPQ
- Filtering improves biological interpretability
- MAPQ thresholds depend on experimental context
- Filtering should happen early in analysis pipelines


#L6: About using MultiQC

## QC Summary with MultiQC

In this section you will learn:
- What MultiQC is and why it is used
- How MultiQC collects alignment QC metrics
- How to compare unfiltered vs filtered BAM files
- How to interpret MultiQC output in the context of mapping quality


## What is MultiQC?

**MultiQC** is a reporting tool that aggregates output from many bioinformatics
programs into a single interactive HTML report.

Instead of manually opening many log files, MultiQC:
- Scans a directory for known output files
- Extracts key QC metrics
- Presents them in a unified report

MultiQC is commonly used for:
- FASTQ quality control
- Alignment statistics
- Library complexity assessment
- Pipeline-wide quality checks


## Why MultiQC matters for mapping quality

Alignment quality information is often spread across tools:
- Bowtie2 reports alignment rates to standard output
- SAMtools reports mapping statistics via `flagstat`
- Filtering changes read counts subtly but significantly

MultiQC allows you to:
- Compare metrics before and after MAPQ filtering
- Detect samples with poor or ambiguous alignments
- Quickly assess whether filtering behaved as expected

This makes MultiQC a critical checkpoint before downstream analysis.


In [ ]:
!pip install -q multiqc

## Generate QC input files

MultiQC can parse output from many tools, including:
- `samtools flagstat`
- Bowtie2 alignment logs

Here we explicitly generate `flagstat` reports for each BAM file.


In [ ]:
%%bash
set -e
cd ~/mapq_demo/alignments

samtools flagstat aligned.sorted.bam > aligned.sorted.flagstat.txt
samtools flagstat aligned.mapq20.bam > aligned.mapq20.flagstat.txt
samtools flagstat aligned.clean.bam > aligned.clean.flagstat.txt

ls -lh *.flagstat.txt


## Run MultiQC

MultiQC scans the current directory and generates a summary report.


In [ ]:
!cd ~/mapq_demo/alignments && multiqc .


This will generate:

*   multiqc_report.html
*   multiqc_data/


## View the MultiQC report

Google Colab cannot display the interactive report inline.
Instead, download the HTML file and open it locally.


In [ ]:
from google.colab import files
files.download('/root/mapq_demo/alignments/multiqc_report.html')


## How to interpret the MultiQC report

Focus on the following sections:

### Alignment summary
- Total reads
- Mapped reads
- Percentage mapped

Compare:
- Unfiltered BAM
- MAPQ-filtered BAM
- Fully cleaned BAM

---

### Effect of MAPQ filtering

You should observe:
- A reduction in total reads after filtering
- Removal of low-confidence (ambiguous) alignments
- Improved reliability of the remaining data

This illustrates why fewer reads can be better than more reads.


## Discussion questions

1. Which BAM file has the highest number of reads?
2. Which BAM file is most trustworthy for downstream analysis?
3. Why is MAPQ = 0 often filtered out?
4. How would MAPQ filtering affect:
   - Coverage tracks?
   - Peak calling?
   - Differential analysis?


## Best practices for using MultiQC

- Run MultiQC at multiple stages of analysis
- Keep both raw and filtered QC reports
- Compare replicates side-by-side
- Always inspect QC before interpretation

MultiQC is often the first thing reviewers request when assessing data quality.


## Key takeaways

- MultiQC aggregates QC metrics across tools and samples
- It provides a global view of alignment quality
- The impact of MAPQ filtering is easy to visualise with MultiQC
- QC inspection is a mandatory step in reproducible genomics workflows
